# アダマールテストを行列で表現  
IBMQでは任意のControlUゲートが作れないので、自分たちで行列表示による量子計算のシュミレートをしようと考えた。  
注意事項: 行列を使って量子状態や演算を表現しているのでビット表示に慣れている人は読みづらい可能性がある。

行列演算の処理にはnumpyを用いる。

In [2]:
import numpy as np
import math

### 制御Uゲートの作成
任意のユニタリ行列UについてCU_gateを作成する。第一ビットが0の時は他ビットは変化しない(単位行列Iが作用する)。第一ビットが1の時は他ビットにUを作用させる。量子ビットに作用させられるUは$2^{n}*2^{n}$行列のみ。

In [3]:
def get_CU_gate(U):
    m,m=U.shape
    n=int(math.log(m,2))
    
    I=np.array([[1,0],[0,1]])
    CU_zero=np.array([[1,0],[0,0]]) # |0><0| 1ビット目は|0>なら|0>を|1>なら|1>を返す
    CU_one=np.array([[0,0],[0,1]]) # |1><1|
    
    for i in range(n):
            CU_zero=np.kron(CU_zero,I) # 1ビット目が|0>ならそれ以降のnビットにはIを作用させる
    
    CU_one=np.kron(CU_one,U) # 1ビット目が|1>ならそれ以降のnビットにはUを作用させる
    
    return CU_zero+CU_one

In [43]:
X=np.array([[0,1],
            [1,0]])
get_CU_gate(X)

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0]])

### アダマールゲートの作成
第一ビットにのみアダマールゲートHを作用させ、その他のビットには変化を加えない(単位行列Iを作用させる)ような行列を作成する。

In [5]:
def get_H_gate(U):
    m,m=U.shape
    n=int(math.log(m,2))
    
    H=np.array([[1,1],[1,-1]])/math.sqrt(2)
    I=np.array([[1,0],[0,1]])
    
    H_gate=H
    
    for i in range(n):
        H_gate=np.kron(H_gate,I)
    
    return H_gate

In [6]:
X=np.array([[0,1],[1,0]])
get_H_gate(X)

array([[ 0.70710678,  0.        ,  0.70710678,  0.        ],
       [ 0.        ,  0.70710678,  0.        ,  0.70710678],
       [ 0.70710678,  0.        , -0.70710678, -0.        ],
       [ 0.        ,  0.70710678, -0.        , -0.70710678]])

### $2^{n}*2^{n}$のユニタリ行列Uと$2^{n}$行$1$列(nビット)の$\phi$を引数としてアダマールテストを行う  
初期状態に、しかるべき演算子を行列の形で左からかけていく。

In [7]:
def hadamard_test(U,phi):
    
    # 初期状態を定める
    zero_bit=np.array([[1],[0]])
    state=np.kron(zero_bit,phi) # 第1ビットは|0>に固定でよい
    
    # stateにH_gateを作用させる
    state=np.dot(get_H_gate(U),state)
    
    # stateにCU_gateを作用させる
    state=np.dot(get_CU_gate(U),state)
    
    # stateにH_gateを作用させる
    state=np.dot(get_H_gate(U),state)
    
    return state # 返すのは終状態

試しにユニタリ行列Xとその固有ベクトルに対してアダマールテストを行う

In [8]:
X=np.array([[0,1],[1,0]])

phi1=np.array([[1],[1]])/math.sqrt(2) # Xの固有ベクトル
phi2=np.array([[1],[-1]])/math.sqrt(2) # Xの固有ベクトル

print(hadamard_test(X,phi1))
print(hadamard_test(X,phi2))

[[0.70710678]
 [0.70710678]
 [0.        ]
 [0.        ]]
[[ 0.        ]
 [ 0.        ]
 [ 0.70710678]
 [-0.70710678]]


### アダマールテストの結果を用いてXの固有値を求めてみる。  
まず終状態の1ビット目で0が測定される確率を求める。  
次にそれを用いて固有値$e^{i\lambda}$に対する$\cos{\lambda}$を求める。  
後はそれを$e^{i\lambda}=\cos{\lambda}+i\sin{\lambda}$に代入してやればいい。

In [9]:
def get_IV_1(state):
    m,k=state.shape
    n=int(math.log(m,2))
    
    PZero=sum([abs(state[i,0])**2 for i in range(2**(n-1))])
    
    cosL=2*PZero-1
    
    # 虚部が正の方を返す
    return cosL+math.sqrt(1-cosL**2)*1j

In [10]:
def get_IV_2(state):
    m,k=state.shape
    n=int(math.log(m,2))
    
    PZero=sum([abs(state[i,0])**2 for i in range(2**(n-1))])
    
    cosL=2*PZero-1
    
    # 虚部が負の方を返す
    return cosL-math.sqrt(1-cosL**2)*1j

実際に試してみる

In [11]:
state1=hadamard_test(X,phi1)
state2=hadamard_test(X,phi2)

print(get_IV_1(state1),get_IV_2(state1))
print(get_IV_1(state2),get_IV_2(state2))

(0.9999999999999991+4.2146848510894035e-08j) (0.9999999999999991-4.2146848510894035e-08j)
(-1+0j) (-1+0j)


Xの固有値と固有ベクトルを実際に計算してみると以下のようになる。  
これより上で求めた固有値はほぼ正しいことが分かる。

In [12]:
def get_answer(U):
    w,v=np.linalg.eig(U)
    
    for i in range(len(w)):
        print('value:',w[i],' vector:',v[:,i])

In [13]:
X=np.array([[0,1],[1,0]])
get_answer(X)

value: 1.0  vector: [0.70710678 0.70710678]
value: -1.0  vector: [-0.70710678  0.70710678]


### 次に初期状態を０ビットに固定してアダマールテストを行う

In [14]:
X=np.array([[0,1],[1,0]])
get_answer(X) # 先に固有値と固有ベクトルを出しておく

value: 1.0  vector: [0.70710678 0.70710678]
value: -1.0  vector: [-0.70710678  0.70710678]


In [15]:
state=hadamard_test(X,[[1],[0]])
get_IV_1(state),get_IV_2(state)

((-4.440892098500626e-16+1j), (-4.440892098500626e-16-1j))

どっちの固有値とも一致しない   
別のユニタリ行列でも試してみる

In [16]:
Z=np.array([[1,0],[0,-1]])
get_answer(Z) # 先に固有値と固有ベクトルを出しておく

value: 1.0  vector: [1. 0.]
value: -1.0  vector: [0. 1.]


In [17]:
state=hadamard_test(Z,[[1],[0]])
get_IV_1(state),get_IV_2(state)

((0.9999999999999991+4.2146848510894035e-08j),
 (0.9999999999999991-4.2146848510894035e-08j))

これは一致してる。

### アダマールテストループ
こんな感じで固有値が求まるかはまちまち。これを解決するためにアダマールテストを何回も繰り返す。これにより終状態が固有ベクトルに収束していく。なお初期状態は$|00...0>$に固定することにする。

In [18]:
def hadamard_loop(U,loop):
    m,m=U.shape
    n=int(math.log(m,2))
    zero=[[1],[0]]
    state=zero
    
    for i in range(n-1):
        state=np.kron(state,zero) # 初期状態を|00...0>にする
    
    for i in range(loop):
        end_state=hadamard_test(U,state)
        # アダマールテストで帰ってきたend_stateから１ビット目の情報を抜き取っていく
        end_state0, end_state1=np.split(end_state,[2**n],axis=0) # end_stateを上下に分ける
        PZero=sum([abs(end_state[i,0])**2 for i in range(2**(n-1))]) # １ビット目で0が観測される確率
        if PZero==0:
            state=end_state1
        else:
            state=end_state0/((PZero)**(1/2)) # 1ビット目の情報を取り除こうとしている。
            
    cosL=2*PZero-1
    cosL=min(cosL,1)
    return cosL+math.sqrt(1-cosL**2)*1j,cosL-math.sqrt(1-cosL**2)*1j # 返すのは固有値(ただし虚部の符号は分からないので両方返す)

In [19]:
X=np.array([[0,1],[1,0]])
get_answer(X) # 先に固有値と固有ベクトルを出しておく

value: 1.0  vector: [0.70710678 0.70710678]
value: -1.0  vector: [-0.70710678  0.70710678]


In [20]:
hadamard_loop(X,1) #ループ一回

((-0.5000000000000002+0.8660254037844385j),
 (-0.5000000000000002-0.8660254037844385j))

In [21]:
hadamard_loop(X,2) #ループ二回

((0.9999999999999991+4.2146848510894035e-08j),
 (0.9999999999999991-4.2146848510894035e-08j))

In [22]:
hadamard_loop(X,3) #ループ三回

((0.9999999999999991+4.2146848510894035e-08j),
 (0.9999999999999991-4.2146848510894035e-08j))

ユニタリ行列Xでは、ループ二回目でほぼ正しい固有値が得られる

In [23]:
CNOT=np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
get_answer(CNOT) # 先に固有値と固有ベクトルを出しておく

value: 1.0  vector: [0.         0.         0.70710678 0.70710678]
value: -1.0  vector: [ 0.          0.         -0.70710678  0.70710678]
value: 1.0  vector: [1. 0. 0. 0.]
value: 1.0  vector: [0. 1. 0. 0.]


In [24]:
hadamard_loop(CNOT,1) #ループ一回

((0.9999999999999991+4.2146848510894035e-08j),
 (0.9999999999999991-4.2146848510894035e-08j))

In [25]:
hadamard_loop(CNOT,2) #ループ二回

((0.9999999999999991+4.2146848510894035e-08j),
 (0.9999999999999991-4.2146848510894035e-08j))

ユニタリ行列CNOTでは、ループ１回目でほぼ正しい固有値が得られる

In [26]:
Y=np.array([[0,-1j],[1j,0]])
get_answer(Y) # 先に固有値と固有ベクトルを確認しておく

value: (0.9999999999999996+0j)  vector: [-0.        -0.70710678j  0.70710678+0.j        ]
value: (-0.9999999999999999+0j)  vector: [0.70710678+0.j         0.        -0.70710678j]


In [27]:
hadamard_loop(Y,1) #ループ一回

((-0.5000000000000002+0.8660254037844385j),
 (-0.5000000000000002-0.8660254037844385j))

In [28]:
hadamard_loop(Y,2) #ループ二回

((0.9999999999999991+4.2146848510894035e-08j),
 (0.9999999999999991-4.2146848510894035e-08j))

In [29]:
hadamard_loop(X,3) #ループ三回

((0.9999999999999991+4.2146848510894035e-08j),
 (0.9999999999999991-4.2146848510894035e-08j))

ユニタリ行列Yもループ二回目でほぼ一致

このようにあらゆるユニタリ行列に対して固有値の推定ができる。

In [30]:
PI=math.pi
e=math.e
T=np.array([[e**(1j*(PI/8)),0],[0,e**(1j*(PI/8))]])
get_answer(T)

value: (0.9238795325112867+0.3826834323650898j)  vector: [1.+0.j 0.+0.j]
value: (0.9238795325112867+0.3826834323650898j)  vector: [0.+0.j 1.+0.j]


In [31]:
hadamard_loop(T,1)

((0.9238795325112859+0.382683432365092j),
 (0.9238795325112859-0.382683432365092j))

In [32]:
hadamard_loop(T,2)

((0.9238795325112863+0.38268343236509084j),
 (0.9238795325112863-0.38268343236509084j))

In [33]:
MyGate=np.kron(X,np.kron(Y,T))
get_answer(MyGate)

value: (0.9238795325112865+0.3826834323650897j)  vector: [ 7.07106781e-01+0.j         -0.00000000e+00+0.j
 -0.00000000e+00+0.j         -0.00000000e+00+0.j
 -0.00000000e+00+0.j         -0.00000000e+00+0.j
  2.77555756e-17+0.70710678j -0.00000000e+00+0.j        ]
value: (-0.9238795325112867-0.38268343236508984j)  vector: [7.07106781e-01+0.j         0.00000000e+00+0.j
 0.00000000e+00+0.j         0.00000000e+00+0.j
 0.00000000e+00+0.j         0.00000000e+00+0.j
 5.55111512e-17-0.70710678j 0.00000000e+00+0.j        ]
value: (0.9238795325112865+0.3826834323650897j)  vector: [-0.00000000e+00+0.j         -0.00000000e+00+0.j
  7.07106781e-01+0.j         -0.00000000e+00+0.j
 -2.77555756e-17-0.70710678j -0.00000000e+00+0.j
 -0.00000000e+00+0.j         -0.00000000e+00+0.j        ]
value: (-0.9238795325112867-0.38268343236508984j)  vector: [ 0.00000000e+00+0.j          0.00000000e+00+0.j
  7.07106781e-01+0.j          0.00000000e+00+0.j
 -5.55111512e-17+0.70710678j  0.00000000e+00+0.j
  0.00000000e+

In [34]:
hadamard_loop(MyGate,1)

((-0.5000000000000002+0.8660254037844385j),
 (-0.5000000000000002-0.8660254037844385j))

In [35]:
hadamard_loop(MyGate,2)

((0.7071067811865466+0.7071067811865485j),
 (0.7071067811865466-0.7071067811865485j))

In [36]:
hadamard_loop(MyGate,3)
# 三回ループでエラーが出てた
# 原因はcosLが1をわずかに超えてしまったこと。 math.sqrt(1-cosL**2)でアウト
# そこでcosLが1を超えたら1にするようにした

((1+0j), (1+0j))

In [37]:
hadamard_loop(MyGate,4)

((0.929788301062429+0.36809470956187595j),
 (0.929788301062429-0.36809470956187595j))

In [38]:
hadamard_loop(MyGate,5)

((0.9178702490927924+0.3968805939200546j),
 (0.9178702490927924-0.3968805939200546j))

In [39]:
hadamard_loop(MyGate,6)

((0.9236410234341965+0.3832587374469499j),
 (0.9236410234341965-0.3832587374469499j))

In [40]:
hadamard_loop(MyGate,7)

((0.9241178848794791+0.3821074912192612j),
 (0.9241178848794791-0.3821074912192612j))

In [41]:
hadamard_loop(MyGate,8)

((0.9238889620238107+0.3826606667147353j),
 (0.9238889620238107-0.3826606667147353j))

In [42]:
hadamard_loop(MyGate,100)

((0.9238795325112867+0.38268343236508984j),
 (0.9238795325112867-0.38268343236508984j))

この値をプロットして収束度合いを可視化しても面白そう